In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

path = "refFlat.txt"
path_fa = "result.fa"

In [2]:
def read_fasta(file_path):
    sequences = {}
    seq_id = None
    current_seq = ""

    with open(file_path, "r") as file:
        for line in file:
            line = line.strip()

            if line.startswith(">"):  
                if seq_id is not None:
                    sequences[seq_id] = current_seq
                    current_seq = ""
                seq_id = line[1:]  
            else:
                current_seq += line

        if seq_id is not None and current_seq != "":
            sequences[seq_id] = current_seq

    return sequences



In [3]:
import re

def split_by_space(text):
    return re.split('\s+', text)

In [4]:
def gene_data(text_list):
    contents =[]
    for i in range(len(text_list)-1):
        contents.append(text_list[i])
    return contents

In [5]:
def result(path):
    with open(path, "r", encoding="utf-8") as file:
        lines =file.readlines()
    result = []
    for line in lines:
        line =split_by_space(line)
        contents=gene_data(line)
        result.append(contents)
    return result
    

In [6]:
gene_data =result(path)
col_names =['geneName','name','chrom','strand','txStart','txEnd','cdsStart','cdsEnd','exonCount','exonStarts','exonEnds']
gene_df=pd.DataFrame(gene_data,columns=col_names)

In [7]:
base_data = read_fasta(path_fa)

In [8]:
def target_base_CT_plus(seq):
    match = re.findall('(?=(\w{20}GG))', seq)
    target_list=[]
    for i in range(len(match)):
        target = match[i][0:3]
        if target =='CAA'or target =='CAG'or target =='CGA':
            target_list.append(True)
        
        else:
            target_list.append(False)
    result =[match[s] for s in range(len(match)) if target_list[s]]
    result = list(dict.fromkeys(result))
    return result

In [9]:
def target_base_AG_plus(seq):
    match = re.findall('(?=(CC\w{20}))', seq)
    target_list=[]
    for i in range(len(match)):
        target = match[i][-3:]
        if target =='TGG':#kここはCCA?
            target_list.append(True)
        
        else:
            target_list.append(False)
    result =[match[s] for s in range(len(match)) if target_list[s]]
    result = list(dict.fromkeys(result))
    return result

In [10]:
def target_start(target_list,qua):    
    result = []
    for i in target_list:
        matches = re.finditer(i, qua)
        for match in matches:
            start_index = match.start()
            result.append(start_index)
    return result

In [11]:
def target_gene_start(name,target):
    data = gene_df[gene_df['name']==name]
    num = data['txStart'].astype(int).to_numpy()
    target = np.array(target)
    result = target+num
    #ここ-1が必要かもしれない？
    return result
    

In [12]:
def exon_data(name,sequence_data):
    data = gene_df[gene_df['name']==name]
    set_num =int(data['txStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    exon_seq_list = []
    start_list = [num for num in start.split(",")]
    end_list = [num for num in end.split(",")]
    for s in range(len(start_list)-1):
        start_num = int(start_list[s])- set_num
        
        end_num= int(end_list[s]) - set_num
    
        
        exon_seq =sequence_data[start_num:end_num]
        exon_seq_list.append(exon_seq)
        
    
    
    return ''.join(exon_seq_list)

In [13]:
def where_start_codon(name):
    data = gene_df[gene_df['name']==name]
    cdsStart =int(data['cdsStart'].to_list()[0])
    set_num =int(data['cdsStart'].to_list()[0])-int(data['txStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    for s in range(len(start)-1):
        if (int(start[s]) <= cdsStart<=int(end[s])):
             exon_num =s
            
    return exon_num


In [14]:
def start_codon(name,exon_data,exon_num):
    data = gene_df[gene_df['name']==name]
    set_num =int(data['cdsStart'].to_list()[0])-int(data['txStart'].to_list()[0])
    CDs =''
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    cds_list=[]
    if exon_num == 0:
            cds_list.append(exon_data[set_num:])
            
    
    
    else:     
        start_num = int(data['cdsStart'].to_list()[0])- int(start[exon_num])

        for s in range(exon_num):
            start_num += int(end[s])-int(start[s])
        cds_list.append(exon_data[start_num:])
        
    return ''.join(cds_list)

In [15]:
def end_codon(cds_seq):
    matches = re.finditer('(?=(CAA)|(?=(CAG))|(?=(CGA))|(?=(TGG)))', cds_seq)
    result=[]
    for match in matches:
        start_pos = match.start() 
        
        if (start_pos % 3) == 0:
            result.append(start_pos)
    return result

In [16]:
#こいつらに要修正

In [17]:
def add_num_list(name):#修正が必要
    data = gene_df[gene_df['name']==name]
    set_num =int(data['cdsStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    add_list=[]
    num = 0
    for s in range(len(start)-1):
        element_list=[]
        element_list.append(num)
        
        num += int(end[s])-int(start[s])
        
        element_list.append(num)
        add_list.append(element_list)
        
    return add_list

In [18]:
def add_num_correct(name,number_list, interval_list,exon_num):
    data = gene_df[gene_df['name']==name]
    set_num =int(data['cdsStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    add_result=[]
    
    number_list = np.array(number_list)
    start_num=int(interval_list[exon_num][0])
    number_list = number_list + (set_num-int(start[exon_num])+start_num)
    number_list = number_list.tolist()#ここでエクソン上で何番目かになってる（cdsからエクソン上に）
    
    
    indices = []
    for number in number_list:
        for index, interval in enumerate(interval_list):
           # print(index)
            if int(interval[0]) <= int(number) < int(interval[1]):
                indices.append(index)
                break
    n =0        
    for i in indices:
        
        if i == 0:
            add_result.append(set_num)
            n +=1
        else:
            add_result.append(int(number_list[n])-int(interval_list[i][0]))
            n += 1

    return add_result

In [19]:
def add_num(name,number_list, interval_list,exon_num):

    data = gene_df[gene_df['name']==name]
    set_num =int(data['cdsStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    add_result=[]
    
    number_list = np.array(number_list)
    start_num=int(interval_list[exon_num][0])
    number_list = number_list + (set_num-int(start[exon_num])+start_num)
    number_list = number_list.tolist()#ここでエクソン上で何番目かになってる（cdsからエクソン上に）

    
    
    indices = []
    for number in number_list:
        for index, interval in enumerate(interval_list):
           # print(index)
            if int(interval[0]) <= int(number) < int(interval[1]):
                indices.append(index)
                break
            
    for i in indices:
        if i == 0:
            add_result.append(set_num)
        else:
            add_result.append(int(start[i]))

    return add_result

In [22]:
gene_df[gene_df['name']=="NM_001301868"]

,geneName,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds
5050,Ikzf1,NM_001301868,chr11,+,11635924,11722926,11650218,11719581,8,"11635924,11650204,11657784,11698310,11704006,1...","11636468,11650258,11657904,11698571,11704174,1..."


In [23]:
#a = exon_data("NM_001301868",base_data['NM_001301868::chr11:11635924-11722926'])

In [24]:
#base_data['NM_001301868::chr11:11635924-11722926'][11723161-11635924:]

In [25]:
#a[5397:]

In [26]:
#start_codon("NM_001301868",a,1)

In [27]:
#CT_target_base_strand_plus("NM_001301868")

In [28]:
def CT_target_base_strand_plus(name,):
    name_df=gene_df[gene_df['name']==name].reset_index()
    chrom = str(name_df.loc[0,'chrom'])
    txStart = str(name_df.loc[0,'txStart'])
    txEnd = str(name_df.loc[0,'txEnd'])
    
    query = f'{name}::{chrom}:{txStart}-{txEnd}'
    seq = base_data[query]
    available_base_num =target_gene_start(name,target_start(target_base_CT_plus(seq),seq))#配列的に狙えるほう
    
    
    
    start_exon_num= where_start_codon(name)
    end_codon_num =end_codon(start_codon(name,exon_data(name,seq),start_exon_num))   
    add_list =add_num(name,end_codon_num,add_num_list(name),start_exon_num)
 
    end_codon_num=add_num_correct(name,end_codon_num,add_num_list(name),start_exon_num)
    
    available_base_num =np.array(available_base_num)
    end_codon_num = np.array(end_codon_num)
    add_list = np.array(add_list)
 
    
    num_able = available_base_num
    num_codon = end_codon_num+add_list

    num_codon = [x for x in num_codon if x <= int(name_df.loc[0,'cdsEnd'])]
    result = np.intersect1d(num_able,num_codon)

    return result

In [29]:
def AG_target_base_strand_plus(name,):
    name_df=gene_df[gene_df['name']==name].reset_index()
    chrom = str(name_df.loc[0,'chrom'])
    txStart = str(name_df.loc[0,'txStart'])
    txEnd = str(name_df.loc[0,'txEnd'])
    
    query = f'{name}::{chrom}:{txStart}-{txEnd}'
    seq = base_data[query]
    available_base_num =target_gene_start(name,target_start(target_base_AG_plus(seq),seq))#配列的に狙えるほう

    
    start_exon_num= where_start_codon(name)
    end_codon_num =end_codon(start_codon(name,exon_data(name,seq),start_exon_num))   
    add_list =add_num(name,end_codon_num,add_num_list(name),start_exon_num)
    
  
    end_codon_num=add_num_correct(name,end_codon_num,add_num_list(name),start_exon_num)
    
    available_base_num =np.array(available_base_num)
    end_codon_num = np.array(end_codon_num)
    add_list = np.array(add_list)

    num_able = available_base_num+19#GGXXXなので19たした
    num_codon = end_codon_num+add_list
    
    num_codon = [x for x in num_codon if x <= int(name_df.loc[0,'cdsEnd'])]
    
    
    result = np.intersect1d(num_able,num_codon)
  
    return result

In [30]:
CT_target_base_strand_plus("NM_001385633")

array([44244686])

In [31]:
CT_target_base_strand_plus("NM_001301868")

array([11650248])

In [32]:
AG_target_base_strand_plus("NM_001385633")

array([], dtype=int32)

In [33]:
AG_target_base_strand_plus("NM_001301868")

array([], dtype=int32)

In [34]:
gene_df[gene_df['geneName']=='Sulf1']

,geneName,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds
77,Sulf1,NM_001198565,chr1,+,12762653,12930596,12856750,12929066,22,"12762653,12798817,12856302,12856691,12866989,1...","12762908,12798912,12856375,12856922,12867229,1..."
78,Sulf1,NM_001198566,chr1,+,12762653,12930596,12856750,12929066,21,"12762653,12856302,12856691,12866989,12875403,1...","12762908,12856375,12856922,12867229,12875555,1..."
79,Sulf1,NM_172294,chr1,+,12788768,12930596,12856750,12929066,22,"12788768,12798817,12856302,12856691,12866989,1...","12789007,12798912,12856375,12856922,12867229,1..."


In [35]:
CT_target_base_strand_plus("NM_001198565")

array([12908728, 12910101, 12918596])

In [36]:
CT_target_base_strand_plus("NM_001198566")

array([12908728, 12910101, 12918596])

In [37]:
CT_target_base_strand_plus("NM_172294")

array([12908728, 12910101, 12918596])

In [38]:
AG_target_base_strand_plus("NM_001198565")

array([12875465, 12906339])

In [39]:
AG_target_base_strand_plus("NM_001198566")

array([12875465, 12906339])

In [40]:
AG_target_base_strand_plus("NM_172294")

array([12875465, 12906339])

In [41]:
gene_df[gene_df['geneName']=='Mul1']

,geneName,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds
30185,Mul1,NM_026689,chr4,+,138161982,138169576,138162070,138166991,4,"138161982,138164938,138165552,138166261,","138162190,138165026,138165673,138169576,"


In [42]:
CT_target_base_strand_plus("NM_026689")

array([138165587, 138166583, 138166796])

In [43]:
AG_target_base_strand_plus("NM_026689")

array([], dtype=int32)

In [45]:
gene_df[gene_df['geneName']=='A1cf']

,geneName,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds
20704,A1cf,NM_001081074,chr19,+,31846212,31929765,31870499,31924910,12,"31846212,31870454,31886191,31888505,31895283,3...","31846259,31870598,31886326,31888636,31895522,3..."
20705,A1cf,NM_001365078,chr19,+,31846212,31929765,31870499,31924910,12,"31846212,31870454,31886191,31888505,31895283,3...","31846259,31870598,31886326,31888636,31895522,3..."


In [46]:
CT_target_base_strand_plus("NM_001365078")

array([31870571])

In [47]:
AG_target_base_strand_plus("NM_001365078")

array([], dtype=int32)